# Second-order wave equation on Chabychev grid with FFT

${\large \frac{\partial^2 u}{\partial t^2}=c^2\frac{\partial^2 u}{\partial x^2}}$

avec $u(-1)=-1, \, u(1)=sin(10t)$



In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

def chebdiff_fft(v):
    N=len(v)-1
    x=np.cos(np.arange(0,N+1)*np.pi/N)
    ii=np.linspace(0,N-1,N,dtype=int)
    V=np.append(v,np.flipud(v[1:-1]))
    U=np.real(np.fft.fft(V))
    f=np.append(ii,np.append(0,-np.flipud(ii[1:])))
    W=np.real(np.fft.ifft(1j*f*U))
    w=np.zeros(N+1)
    w[1:-1]=-W[1:N]/np.sqrt(1-x[1:-1]**2)
    w[0]=np.sum(ii**2*U[ii])/N+0.5*N*U[N]
    w[-1]=np.sum((-1)**(ii+1)*ii**2*U[ii])/N+0.5*(-1)**(N+1)*N*U[N]
    return(w)



In [13]:
#vitesse de phase
c=5
#taille de la corde
L=3
#fréquence d'excitation
n=1.5
omg=n*c*np.pi/L
#temps d'intégration
tmax=5

In [14]:
N=80
x=np.cos(np.arange(0,N+1)*np.pi/N)
dt=(8/N**2)/10

u=np.zeros(N+1)
M=int(np.round(tmax/dt))+1
t=np.linspace(0,tmax,M)
Dt=100*dt
Mspan=int(np.round(tmax/Dt))+1
tspan=np.linspace(0,Dt*((Mspan)-1),Mspan)

U=np.zeros((N+1,Mspan))
ii=0
T=0


u=np.zeros(N+1)
u_old=np.zeros(N+1)
for i in range(M):
    if t[i]>=T:
        U[:,ii]=u
        ii=ii+1
        T=Dt*ii
    w=chebdiff_fft(chebdiff_fft(u))
    w[0]=0
    w[-1]=0
    u_new=2*u-u_old+dt**2*c**2*w
    u_new[0]=0 #0.01*np.sin(3*c*np.pi/2*t[i])
    u_new[-1]=0.01*np.sin(omg/(2/L)*t[i])
    u_old=u
    u=u_new
    


In [15]:
x=np.cos(np.arange(0,N+1)*np.pi/N)
x_plt=(x+1)*(L/2)

xx,ttspan=np.meshgrid(x_plt,tspan)
fig,ax=plt.subplots(1,2,figsize=(15,7))
a=ax[0].contourf(xx,ttspan,U.T,cmap='viridis')
a=ax[1].contourf(xx,ttspan,U.T,cmap='viridis')
ax[1].set_ylim(tmax-5,tmax)
ax[0].set_xlabel('x',fontsize=20)
ax[0].set_ylabel('t',fontsize=20)
ax[0].tick_params('both',labelsize=20)
ax[1].set_xlabel('x',fontsize=20)
ax[1].set_ylabel('t',fontsize=20)
ax[1].tick_params('both',labelsize=20)
fig.colorbar(a)
fig.suptitle(r'L=%2s, c=%2s et $\omega_{0}$=%1s $c \frac{\pi}{L}$'%(L,c,n),fontsize=20)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'L= 3, c= 5 et $\\omega_{0}$=1.5 $c \\frac{\\pi}{L}$')

In [16]:
from mpl_toolkits.mplot3d import Axes3D

xx,ttspan=np.meshgrid(x_plt,tspan)


fig,ax=plt.subplots(figsize=(10,10),subplot_kw=dict(projection='3d'))
plt.set_cmap('jet_r')

ax.plot_surface(xx,ttspan,U.T,cmap='viridis')

ax.set_zlim(-0.1,0.1)
ax.set_xlabel('x')
ax.set_ylabel('t')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …